In [21]:
import numpy as np
from numpy.linalg import eigvals
import matplotlib.pyplot as plt
%matplotlib qt5
#
#  extend path by location of the dvr package
#
import sys
sys.path.append('/home/thomas/Python')
import dvr

In [2]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [19]:
#
#  default 1D potential has a resonance just below 7 eV
#
def Jolanta_1D(x, a=0.2, b=0.0, c=0.14):
    return (a*x*x-b)*np.exp(-c*x*x)

#
#  default 3D potential has a resonance at 1.75 eV - 0.2i eV
#
def Jolanta_3D(r, a=0.1, b=1.2, c=0.1, l=1):
    return (a*r**2-b)*np.exp(-c*r**2) + 0.5*l*(l+1)/r**2

In [32]:
#
#  compute DVR of T and V
#  then show the density of states
#  in a potential + energy-levels plot
#
rmin=0
rmax=25      # grid from 0 to rmax
thresh = 5   # maximum energy for state in the plot in eV
ppB = 10     # grid points per Bohr

nGrid=int((rmax-rmin)*ppB)
print("nGrid = %d" % nGrid)
rs = dvr.DVRGrid(rmin, rmax, nGrid)
Vs = Jolanta_3D(rs, a=0.1, b=1.2, c=0.1, l=1)
Ts = dvr.KineticEnergy(1, rmin, rmax, nGrid)
[energy, wf] = dvr.DVRDiag2(nGrid, Ts, Vs)

n_ene=0
for i in range(nGrid):
    print("%3d  %12.8f au = %12.5f eV" % (i+1, energy[i], energy[i]*au2eV))
    n_ene += 1
    if energy[i]*au2eV > thresh:
        break

# "DVR normalization", sum(wf[:,0]**2)
# this is correct for plotting

c=["orange", "blue"]
#h=float(xmax) / (nGrid+1.0)
scale=150

plt.plot(rs,Vs*au2eV, '-', color="black")
for i in range(n_ene):
    plt.plot(rs, scale*wf[:,i]**2+energy[i]*au2eV, '-', color=c[i%len(c)])
plt.ylim(energy[0]*au2eV-1, energy[n_ene-1]*au2eV+1)
plt.xlabel('$r$ [Bohr]')
plt.ylabel('$E$ [eV]')
plt.show()

nGrid = 250
  1    0.01724346 au =      0.46922 eV
  2    0.05016919 au =      1.36517 eV
  3    0.07122177 au =      1.93804 eV
  4    0.12020844 au =      3.27104 eV
  5    0.19775190 au =      5.38110 eV


In [31]:
""" complex diagonalization example """

theta=40.0/180.0*np.pi

Vs = Jolanta_3D(rs*np.exp(1j*complex(theta)), a=0.1, b=1.2, c=0.1, l=1)
H_theta = np.exp(-2j*complex(theta)) * Ts + np.diag(Vs)
energies = eigvals(H_theta)
energies.sort()
energies[:10]*au2eV

array([-0.38910817 -8.32625185j, -0.3036125  -5.19717426j,
       -0.17740745-12.46080023j, -0.13653951 -2.9155987j ,
        0.00176073 -1.35626655j,  0.04015234 -0.43194784j,
        1.01304107-17.26687205j,  1.35860173-27.72629193j,
        1.38337959-21.07944233j,  1.75699175 -0.19928624j])

In [ ]:
#
# above theta = 40 deg the lowest Re(E) are basis set 
# representation artifacts, and we should either not go there
# or use a better filter for the states to keep
#

In [37]:
#
#  complex scaling loop: 
#
#  start on the real axis (theta=0) and rotate to theta = theta_max 
#
#  we keep n_keep energies with the lowest real part 
#

n_theta=50
n_keep=15
theta_min=0
theta_max=40.0/180.0*np.pi

thetas=np.linspace(theta_min, theta_max, n_theta, endpoint=True)
run_data = np.zeros((n_theta,n_keep), complex)  # array used to collect all theta-run data

for i_theta in range(n_theta):
    theta=thetas[i_theta]
    Vs = Jolanta_3D(rs*np.exp(1j*complex(theta)), a=0.1, b=1.2, c=0.1, l=1)
    H_theta = np.exp(-2j*complex(theta)) * Ts + np.diag(Vs)
    energies = eigvals(H_theta)
    energies.sort()
    run_data[i_theta,:] = energies[0:n_keep]
    print(i_theta+1, end=" ")
    if (i_theta+1)%10==0:
        print()

run_data *= au2eV

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 


In [41]:
#
# even states in blue, odd states in yellow
#
for i in range(0, n_keep, 2):
    plt.plot(run_data[:,i].real,  run_data[:,i].imag, 'o', color='blue')
plt.xlim(0,5)
plt.ylim(-1,0)
plt.show()

In [10]:
#
# You have to hand-select the L-ranges for your choice of a, b, c,
# and grid parameters
#
# I suggest you put the crossing_j.dat files into a directory 
# with a useful name
#

In [67]:
#
#  save n_save trajectories to file
#
n_save=5
fname="complex_scaling_rmax."+str(int(rmax))+"_ppB."+str(ppB)+".csv"
tr = np.zeros((n_theta,n_save+1), complex)
tr[:,0]=thetas
tr[:,1:n_save+1]=run_data[:,0:n_save]
header="theta"
for i in range(n_save):
    header = header + ', E' + str(i+1)
np.savetxt(fname, tr, fmt='%15.12f', delimiter=',', header=header, comments='')